In [ ]:
%pip install langchain langchain-community langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.7 MB/s eta 0:00:00
 

In [ ]:
%pip install -U langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.9 MB/s eta 0:00:00
ERROR: Operation cancelled by user


In [ ]:
%pip install gpt4all

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.6 MB/s eta 0:00:00


In [ ]:
%pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 2.7 MB/s eta 0:00:00


In [ ]:
import getpass
import os

os.environ["GROQ_API_KEY"] = getpass.getpass()

from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

··········


In [ ]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

··········


In [ ]:
import pandas as pd

df = pd.read_csv("o1_labels.csv")

texts = df['text_translated'].tolist()

In [ ]:
print(len(texts))

313


In [ ]:
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
import torch
from langchain.embeddings import GPT4AllEmbeddings
from langchain.docstore.document import Document

texts = texts[:20]
print(len(texts))

docs = [Document(page_content=text) for text in texts]
print(len(docs))

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
print(len(splits))

# Create the vectorstore
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=GPT4AllEmbeddings()
)
# vectorstore = Chroma(
#     collection_name=splits,
#     embedding_function=embeddings  # Where to save data locally, remove if not necessary
# )
# vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("What is RTI"):
    print(chunk, end="", flush=True)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


20
20
2527


Downloading: 100%|██████████| 45.9M/45.9M [00:01<00:00, 26.1MiB/s]
Verifying: 100%|██████████| 45.9M/45.9M [00:00<00:00, 556MiB/s]


RTI stands for Research, Technology, and Innovation.

In [ ]:
for chunk in rag_chain.stream("How does Spanish government address on gender equality"):
    print(chunk, end="", flush=True)

The Spanish government addresses gender equality in AI through funding programs that promote increasing the share of female AI developers and initiatives for networking and coaching for women in AI. They also set standards to improve professional qualifications, aiming to reduce the gender pay gap. Additionally, they aim to increase the proportion of female talent in the AI ecosystem through sustainable support programs and scholarships.

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "What reforms are being made in education systems to prepare citizens for the future workforce?"})
print(response["answer"])

The Australian government has been investing in education and skills training to prepare citizens for the future workforce. Reforms include the implementation of the Australian Curriculum, which includes digital literacy and digital technologies, and the development of targeted training courses and microcredentials to build digital skills. Additionally, the government is reviewing the curriculum to ensure a focus on essential content and core skills.


In [ ]:
response = rag_chain.invoke({"input": "What policies does French government implement?"})
print(response["answer"])

I don't know. The provided context does not mention French government policies. It appears to focus on Austria, discussing its reform plans, tax reforms, and national reform programme.


In [ ]:
texts2 = df['text_translated'].tolist()
texts2 = texts2[20:60]
docs2 = [Document(page_content=text) for text in texts2]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits2 = text_splitter.split_documents(docs2)
vectorstore.add_documents(documents=splits2)

KeyboardInterrupt: 

In [ ]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "What policies does Canada government implement?"})
print(response["answer"])

In [ ]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
import torch
from langchain.embeddings import GPT4AllEmbeddings
from langchain.docstore.document import Document

docs = [Document(page_content=text) for text in texts]


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)


embeddings = GPT4AllEmbeddings()
vector_store = Chroma.from_documents(
    documents=splits,
    embedding=GPT4AllEmbeddings(),
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)


# vectorstore = Chroma(
#     collection_name=splits,
#     embedding_function=embeddings  # Where to save data locally, remove if not necessary
# )
# vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("What is RTI"):
    print(chunk, end="", flush=True)

RTI stands for Research, Technology, and Innovation.

In [ ]:
texts2 = df['text_translated'].tolist()
texts2 = texts2[20:60]
docs2 = [Document(page_content=text) for text in texts2]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits2 = text_splitter.split_documents(docs2)
vectorstore.add_documents(documents=splits2, persist_directory="./chroma_langchain_db")

['faf8a6ae-bb2d-4cf2-9fa8-7f21c0eff5ce',
 '25879ebc-5e63-4c90-b4a3-5cca841d3af8',
 '382c7782-23e3-42b4-a05e-77841c79fc5d',
 '72ea2eb2-677c-45c9-abd5-01996c744272',
 'ceee709f-5740-4da7-bf0e-7a0b5f11ca79',
 '5fad3b56-ba5f-402d-b841-ebc1503dbea8',
 '96a8a984-7ba5-4fc6-b19e-ffe6cd27c62f',
 '93d56a73-9eca-43d4-a4bd-1b8e2c03c658',
 '2c0dca5a-a302-4949-83b8-542864ba0e79',
 '9fa12c61-893a-4f8f-9a21-6a8b267f7ea5',
 '44758a34-c0df-41ba-9d49-969ba34cced3',
 '57ef6ec7-c08f-4c5d-9d1e-6bc9ddf829cc',
 'f2427b8f-5569-417f-aaef-353ec1dd42aa',
 '18203447-cca4-4d47-9f81-6c0c7c3ac26d',
 'b78e0ddd-bc62-4147-9c08-5fb44b4369af',
 '07d2f828-cc49-42e2-b6f5-c5a9f53081d8',
 '4f367730-6e66-4b00-8bc6-e2766331d2da',
 '25bbc9a6-5d61-493f-b73f-ec28f71804fb',
 '9d062435-0fe3-4b71-b40f-c4fc69e1890c',
 'd9b59e73-1c33-45c4-8c69-a1e304944453',
 'dcb694be-1fbb-4f37-8cf6-a576807607f8',
 '157afc0e-645d-4ba3-87c4-5c0ef32fe194',
 'a9eb5ca4-f68c-48b3-a39e-48a2024b531d',
 'd272aa40-2ca3-4ab6-9356-ae4f171a44ef',
 'ba253b93-540e-

In [ ]:
print(len(texts2))

40


In [ ]:
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("What policies does Canada government implement?"):
    print(chunk, end="", flush=True)

Based on the provided context, the Canadian government implements policies such as Government Operations, Fairness and Openness, Supporting Service Delivery On Reserve, Government of Canada IT Infrastructure, and policies to fight COVID-19 and support people.

In [ ]:
for chunk in rag_chain.stream("What policies does Switzerland government implement?"):
    print(chunk, end="", flush=True)

The Swiss government implements policies through a comprehensive approach, focusing on priority areas such as sustainable development, social cohesion, and international cooperation. The Federal Council adopts a range of measures to achieve these objectives, including reducing discrimination, promoting democracy, and aligning with the 2030 Agenda. Additionally, the government ensures that its policies take into account the interests of all age groups and respect the intergenerational pact.

In [ ]:
for chunk in rag_chain.stream("What strategies are in place to transition to renewable energy sources, specify the strategy and its country name"):
    print(chunk, end="", flush=True)

The strategy to transition to renewable energy sources in Belgium involves increasing the production capacity of electricity, heat, and cold from renewable sources, as well as supporting the production of electricity from renewable sources through the revision of the green certificates mechanism. The country aims to produce 10 TWh of renewable electricity and 14.2 TWh of renewable heat by 2030. Additionally, the government will promote the establishment of renewable energy throughout the territory, from the North Sea to the German-speaking region.

In [ ]:
for chunk in rag_chain.stream("What policies does Austria implement?"):
    print(chunk, end="", flush=True)

Austria implements policies such as regulating the sharing economy and online trading, greening the tax system, and reducing payroll and income tax. The country also has a focus on making Austria more attractive, increasing competitiveness, and positioning it as a digital innovation region. Additionally, Austria has a national reform plan and a digital action plan that outlines measures to achieve specific goals.

In [ ]:
all_texts = df['text_translated'].tolist()


In [ ]:
texts3 = all_texts[60:100]
docs3 = [Document(page_content=text) for text in texts3]
print(len(docs3))
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits3 = text_splitter.split_documents(docs3)
print(len(splits3))
vectorstore.add_documents(documents=splits3, persist_directory="./chroma_langchain_db")

40
8456


['374a4ef5-f2fa-4ad9-b986-ab5343c15d35',
 '371ee844-999c-4291-9022-f4b8f97e7d9f',
 '9a5a4e8f-471e-4115-8d06-79072ef3edff',
 '841f1725-e425-4847-b598-21805ad67e6d',
 '45bc0148-3b52-4d46-990f-e25daefb297a',
 'e66f7b55-f88a-4445-971d-bba26c72a331',
 'e7e711c4-74b9-42b3-9767-5d55d14fe1a3',
 '8fd30361-47c6-4647-b691-945808d7ede3',
 'd27ea207-7680-4126-82e7-4f92f04b0437',
 '1973262a-992b-4eed-8b4e-02c06f2f5f05',
 '7194ae21-9961-405c-9098-2276f82a985c',
 '516a5096-785c-40f8-9521-48a09fd1865c',
 'ce2a6b6b-2f2a-43ee-8946-17c90745a80b',
 '256755bf-a56a-4226-b66e-0ecd70465d39',
 'b8a12d00-eca9-410f-8726-c1016c80717f',
 'ec1551e6-5054-4589-91bf-2966af9be44e',
 '5261c1b9-e016-4d33-8905-58e7d281bd5c',
 'caca9312-facd-47d6-83c6-88d0ba7fda83',
 'f111d097-3e53-41df-9c23-104be5c7296c',
 '3cdb00d1-31d0-4ebf-ab19-55cd78cdca89',
 'e816cbb8-d443-48e4-9ad8-f08119d5d1d4',
 '35c1ca3c-f6f3-41da-88a7-1d2cddfe9306',
 '0e0307d5-513a-42da-81c6-c75cd9435c10',
 '9593f574-0259-49d3-a411-f0fe8ce8da1e',
 '4e10f95f-ae8e-

In [ ]:
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("What policies does Costa Rica government implement?"):
    print(chunk, end="", flush=True)

The Costa Rican government implements policies such as the National Policy of Society and Economy Based on Knowledge, which aims to promote entrepreneurial and innovative attitudes, and the Sustainable Development Goals (SDGs), which focus on reducing poverty, mitigating the impact of carbon emissions, and promoting development. Additionally, the government has established a regulatory framework to promote the development of small and medium-sized enterprises (SMEs) since 2002.

In [ ]:
for chunk in rag_chain.stream("What policies does French government implement?"):
    print(chunk, end="", flush=True)

The French government implements policies in various areas, including management of the health crisis, healthcare, social security, pensions, state and civil service, poverty alleviation, and more.

In [ ]:
texts4 = all_texts[100:150]
docs4 = [Document(page_content=text) for text in texts4]
print(len(docs4))
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits4 = text_splitter.split_documents(docs4)
print(len(splits4))
vectorstore.add_documents(documents=splits4, persist_directory="./chroma_langchain_db")

50
19407


['8237903a-bfd7-4ec9-9408-fed6c639c92a',
 '77aca093-ce09-4ca0-a45b-fd23992f524c',
 '11c299e8-47fa-4788-8fe8-79ff352a3362',
 '46072461-6e82-4226-becc-bd2c85669b64',
 'b14d6600-3efc-4a46-8e2c-3fffb89239e6',
 '92d332bc-a610-446e-a826-4ddbec8cbb0e',
 'db95afae-f4d1-4251-83c1-5ca5875dd66d',
 '204209d2-ff83-4ac9-a902-79c7e02bb627',
 '4eba6f55-9320-4b82-a279-a1ebccf194a7',
 '371a5928-c223-4f4e-9cee-3be3325a37b9',
 '69c1e6dd-8478-47f8-823f-c1f7b0f037dd',
 '5bc61842-32b4-4f0f-bad1-0e458ccdee0f',
 '7ce2fa21-f417-418f-8ed0-37e1a5523e7b',
 '6f955604-8f8c-4ed8-8de1-57d5b882a62c',
 '58e7e173-738d-4453-85ef-afbe44673bce',
 '71a3b677-d957-42aa-b4a8-7b69c7e8e251',
 'a6517890-d4e8-440b-bc70-d190db12914a',
 '7d3a42ef-12b2-4d33-9859-11d74ba104cd',
 '9c951cfa-c328-4c70-9903-2292f4f5cf04',
 '32449ffe-fe35-44a2-9fa1-efb7c0df9da5',
 'ff24ac7d-11b9-4efe-a4c7-b8daf1d6b884',
 'c80fca72-0b9e-43bc-b804-f3a36e460e6a',
 '72e4a78f-db7c-40b2-ba40-f646b9a43084',
 '89753d3b-7720-4c44-8254-9476e1945a1a',
 'e4d3484e-bd76-

In [ ]:
texts5 = all_texts[288:]
docs5 = [Document(page_content=text) for text in texts5]
print(len(docs5))
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits5 = text_splitter.split_documents(docs5)
print(len(splits5))
vectorstore.add_documents(documents=splits5, persist_directory="./chroma_langchain_db")

25
9633


['2f150c45-1158-4263-a87f-931d68e77f35',
 'c8679b2c-3f82-47e4-a74c-fcf104f559d5',
 'b5bb3aa1-0da6-40d1-a41e-b3207bbeb381',
 '19a88d15-26a4-48e7-a55f-4b959298033d',
 '63967912-17e0-416a-877b-0567c2d5ec73',
 'aa1535c3-c6fd-4714-b8e5-37cefaec928b',
 '053e164a-b5e3-4440-bd22-e08517b1e5d4',
 '4bfeebe4-ef24-4a14-b8b0-7e46e8a57d0f',
 '6f02e3f6-6302-4a9d-b18e-dedc45b1e138',
 'e3d61016-d9a5-4b0d-9d2f-be0d4d25dbd5',
 'e6c743dc-fce6-4e12-8b1e-ab36c8db0a30',
 'c0657104-9791-4e69-a158-e04999c3275d',
 'f6a6e4ec-9ca5-4fd0-9353-741e5654d72c',
 '675de39d-5456-4909-acff-3f5c247857f2',
 'c731aca2-92c2-4c04-be7a-6fd57c1169bf',
 'd24c8068-6629-4991-9d72-2b1361dd9d9c',
 'a72f37e0-14c9-4de3-9109-6d306e279036',
 '4e65970c-4b1b-4e37-9403-da0e6c06b2e7',
 'a03f4070-951f-401e-bfbc-47d71d9fb708',
 '1c5d938d-30ba-45b1-b5e6-c733cea6434c',
 'da0f89db-5a8e-4e69-9b6f-d93d4d4a07a3',
 '168a8300-f78f-4522-b72e-acb4971c0cb3',
 '4b13fae0-9620-4410-a5a5-22637ef6055d',
 '759a9c60-a810-4e53-a366-4973c1a79658',
 '53f8f924-c5d8-

In [ ]:
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
questions = [
    "What is RTI?",
    "In United State, What strategies are being implemented to support small and medium-sized enterprises?",
    "In United State, how do they adapting their industrial policies to thrive in a digital economy?",
    "In United State, How are national policies addressing climate change and reducing carbon emissions?",
    "In United State, What strategies are in place to transition to renewable energy sources?",
    "In United State, How are gender equality and equal opportunities being advanced?",
    "In United State, How are national policies improving healthcare accessibility and quality?",
    "In United State, What reforms are being made in education systems to prepare citizens for the future workforce?"
    "Summarize the main points from the documents in United State, ."
]

for question in questions:
  for chunk in rag_chain.stream(question):
    print(chunk, end="", flush=True)
  print("\n")

RTI stands for Research, Technology, and Innovation.

To support small and medium-sized enterprises (SMEs) in the United States, strategies include supporting promising small businesses to scale their businesses, addressing capital needs for growth, and connecting them to the commercial value chain. The Small Business Administration (SBA) and the Export-Import Bank (EXIM) are implementing these measures to support SMEs.

The United States is adapting its industrial policies to thrive in a digital economy by implementing measures such as data protection, digital infrastructure, and policies that encourage digital technology development. These measures support the growth of domestic firms and attract foreign direct investment.

According to the context, the United States is not mentioned, but the context refers to the Swiss government's plan to achieve carbon neutrality by 2035. The plan includes emission reduction targets for 2030 and 2040, and the implementation of climate plans to ach

In [ ]:
for chunk in rag_chain.stream("What policy tools are mentioned in the national strategies? "):
    print(chunk, end="", flush=True)

The policy tools mentioned in the national strategies are not explicitly stated in the provided context. However, the context does mention that the Federal Council is undertaking priority domestic policy measures to achieve its objectives for 2030, and that these measures are presented in an action plan.

In [ ]:
for chunk in rag_chain.stream("What is the main difference of policies regarding to energy in America and Canada"):
    print(chunk, end="", flush=True)

Based on the provided context, the main difference in policies regarding energy in America and Canada is that Canada has a stronger focus on clean energy and reducing emissions, with a goal of generating 82% of its electricity from non-emitting sources. In contrast, the United States has a more fragmented approach to energy policy, with states like California and British Columbia taking the lead on clean fuel standards.

In [ ]:
for chunk in rag_chain.stream("What are the countries that specifically address on education?"):
    print(chunk, end="", flush=True)

Based on the provided context, the countries that specifically address education are not explicitly mentioned. However, the context does mention the importance of education in achieving goals such as reducing poverty and promoting gender equality, and highlights the impact of education on regional development and social cohesion.

In [ ]:
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
# from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
import torch
from langchain.embeddings import GPT4AllEmbeddings
from langchain.docstore.document import Document

texts = texts[288:]
print(len(texts))

docs = [Document(page_content=text) for text in texts]
print(len(docs))

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
print(len(splits))

# Create the vectorstore
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=GPT4AllEmbeddings()
)
# vectorstore = Chroma(
#     collection_name=splits,
#     embedding_function=embeddings  # Where to save data locally, remove if not necessary
# )
# vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
labels = [
    "Net zero transitions policy debates",
    "Government capabilities for net zero transitions",
    "Net zero transitions in transport and mobility",
    "Net zero transitions in food and agriculture",
    "Cross-sectoral policies for net zero",
    "Digital transformation of research-performing organisations",
    "Open and enhanced access to publications",
    "Open and enhanced access to research data",
    "Research security",
    "STI plan or strategy",
    "Strategic policy intelligence",
    "Evaluation and impact assessment",
    "Public research strategies",
    "Competitive research funding",
    "Non-competitive research funding",
    "Research integrity and reproducibility",
    "Structural change in the public research system",
    "High-risk high-reward research",
    "Research and technology infrastructures",
    "Internationalisation in public research",
    "Cross-disciplinary research",
    "Third-party funding",
    "Business innovation policy strategies",
    "Financial support to business R&D and innovation",
    "Non-financial support to business R&D and innovation",
    "Stimulating demand for innovation and market creation",
    "Entrepreneurship capabilities and culture",
    "Targeted support to SMEs and young innovative enterprises",
    "Foreign direct investment",
    "Access to finance for innovation",
    "Knowledge exchange and co-creation strategies",
    "Collaborative research and innovation",
    "Commercialisation of public research results",
    "Inter-sectoral mobility",
    "Intellectual property rights in public research",
    "Cluster policies",
    "STI human resources strategies",
    "STEM skills",
    "Doctoral and postdoctoral researchers",
    "Research careers",
    "Equity, diversity and inclusion (EDI)",
    "International mobility of human resources",
    "Research and innovation for society strategy",
    "Research and innovation for developing countries",
    "International STI governance policy",
    "Multi-stakeholder engagement",
    "Science, technology and innovation culture",
    "Digital transformation of firms",
    "Ethics of emerging technologies",
    "Horizontal policy coordination",
    "Mission-oriented innovation policies",
    "Net zero transitions in energy"
]

# Join the labels into a single string separated by newlines
formatted_string = "\n".join(labels)

# Print the organized string
print(formatted_string)
question = "determine the relevant labels mentioned in U.S. document policy" + formatted_string
for chunk in rag_chain.stream(question):
    print(chunk, end="", flush=True)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


25
25
9633


Downloading: 100%|██████████| 45.9M/45.9M [00:01<00:00, 40.6MiB/s]
Verifying: 100%|██████████| 45.9M/45.9M [00:00<00:00, 433MiB/s]


Net zero transitions policy debates
Government capabilities for net zero transitions
Net zero transitions in transport and mobility
Net zero transitions in food and agriculture
Cross-sectoral policies for net zero
Digital transformation of research-performing organisations
Open and enhanced access to publications
Open and enhanced access to research data
Research security
STI plan or strategy
Strategic policy intelligence
Evaluation and impact assessment
Public research strategies
Competitive research funding
Non-competitive research funding
Research integrity and reproducibility
Structural change in the public research system
High-risk high-reward research
Research and technology infrastructures
Internationalisation in public research
Cross-disciplinary research
Third-party funding
Business innovation policy strategies
Financial support to business R&D and innovation
Non-financial support to business R&D and innovation
Stimulating demand for innovation and market creation
Entrepreneur